# Music recommender


### Import csv created with top 100 Billboard songs

In [1]:
# Load the dataset
import pandas as pd
import random
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from IPython.display import IFrame


#Initialize SpotiPy with user credentials: this is a popular way of ensuring that your credentials are safe
#I am not pushing anything to Github!!! I am only storing my secret credentials in a .py file outside, but in the same
#folder in my computer.
#What can be done is creating a git_ignore file to indicate to github that it does not have to upload it.
#Before commiting it, I can right click and select ignore file.

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

#Improting df´s
#  1 . Hot songs
url = 'hot100_songs.csv'
hot100_df = pd.read_csv(url, index_col=[0])

#  2.  Cleaned numerical for model creation, to use columns in program
cleaned_df = pd.read_csv("cleaned_audiofeatures.csv", index_col=[0])

#  3.  With cluster column
clustered_df = pd.read_csv("clustered_audiofeatures.csv", index_col=[0])

#  4.  With id´s of songs to be able to recommend
ids_df = pd.read_csv("with_ids.csv", index_col=[0])

#  5.  Big DF with all downloaded songs
titles_df = pd.read_csv("final_music_data.csv", index_col=[0])


In [11]:
from difflib import get_close_matches

# Function to find the closest match in a list of choices
def get_closest_match(input_str, choices):
    match = get_close_matches(input_str, choices, n=1, cutoff=0.8)
    return match[0] if match else None

#Function to load picke files
def load(filename = "scaler_model_songrecommender.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
    except FileNotFoundError: 
        print("File not found!") 

#function to display song
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

favorite_song = input("Please enter your favorite song:\n\n").lower()

# If there is an exact match (0 typos) between input song and a song in the hot100 , display another one randommly from that 100hot list
exact_match = hot100_df[hot100_df["Title"].str.lower() == favorite_song]
if not exact_match.empty: #The if not exact_match.empty condition checks if the DataFrame exact_match is not empty.
    rec_song = random.choice(hot100_df["Title"])
    results_for_users= sp.search(q=rec_song)
    track_id = results_for_users["tracks"]["items"][0]["id"]
    print(f"\n\nAmazing tune! It is in our 100 Hot Songs of the year list.\n You would also like listening to this other hit: \n\n ")
    display(play_song(track_id))

# Check for a close match if there's a typo
else:
    closest_match = get_closest_match(favorite_song, hot100_df["Title"].str.lower().values)
    if closest_match:
        potential_matches = hot100_df[hot100_df["Title"].str.lower() == closest_match]
        if not potential_matches.empty:
            print(f"\n\nDid you mean '{closest_match}'?")
            print("Here are some potential matches with different authors:")
            for idx, (index, row) in enumerate(potential_matches.iterrows(), start=1): #printing index of the nº of results found
                print(f"{idx}. {row['Title']} by {row['Artist']}")

            user_choice = input("\n\nEnter the number of the song you meant, or enter 'N' if none of them:\n")
            #to check if user inputs consists of digits (bolean result) only AND is between 1 and length of potential_matches found
            if user_choice.isdigit() and 1 <= int(user_choice) <= len(potential_matches):
                rec_song = random.choice(hot100_df["Title"])
                results_for_users= sp.search(q=rec_song)
                track_id = results_for_users["tracks"]["items"][0]["id"]
                print(f"\n\nCool song!\n\n Try listening to this other hit.\n You might like it.\n\n")
                display(play_song(track_id))

            else:
                print("Okay, no worries! If you have another song in mind, feel free to enter it.")
        else:
            print(f"Unfortunately, no matching songs were found in our top 100 list!\n")
    else:
        print(f"\n\nUnfortunately, that song is not currently in our top 100 list.\n\n I can still recommend something you might like.\n")
        #looking for the acustic features of the song input by user, I need to first get the id
        fs_details = sp.search(favorite_song)
        id = fs_details['tracks']['items'][0]['id']
        
        #instead of looping over them - too many requests and get timeout by spotify - do this:
        fs_features = sp.audio_features(id)[0]
        inputfs_df = pd.DataFrame(fs_features, index=[0], columns = cleaned_df.columns)
        
        scaler= load(filename="scaler_model_songrecommender.pickle")
        scaler_final = scaler.transform(inputfs_df)
        scaled_df = pd.DataFrame(scaler_final, columns = cleaned_df.columns)
        kmeans= load(filename="ModelSongRecommender.pickle")
        cluster_favsong = kmeans.predict(scaled_df)
        
        clustered_df_filtered = clustered_df[clustered_df["cluster"] == cluster_favsong[0]]
        
        if not clustered_df_filtered.empty:
            rec_song = random.choice(clustered_df_filtered.index)
            results_for_users = sp.search(q=rec_song)
            track_id = results_for_users["tracks"]["items"][0]["id"]
            print("\n\n What do you think about this other hit: \n\n")
            display(play_song(track_id))
            
        else:
            print("No matching songs found!")



Please enter your favorite song:

 boogie wonderland




Unfortunately, that song is not currently in our top 100 list.

 I can still recommend something you might like.



/Users/esthervelasco/anaconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/esthervelasco/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/esthervelasco/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/esthervelasco/anaconda3/lib/python3.11/site-pa



 What do you think about this other hit: 


